# Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st

In [3]:
# Load the data:
pokemon = pd.read_csv("Pokemon.csv")
pokemon.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [4]:
# Your code here

t1 = pokemon["Type 1"].unique()
t2 = pokemon["Type 2"].unique() 
unique_types = set(np.append(t1, t2))
len(unique_types) # you should see 19

19

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [5]:
pokemon_totals = []
ppk = {}
# Your code here
for i in unique_types:
    if type(i) == float:
        pass
    else:
        a = pokemon[pokemon["Type 1"]==i].value_counts()
        
        b = pokemon[pokemon["Type 2"]==i].value_counts()
        tt = len(a) + len(b)
        ppk[i] = tt
        pokemon_totals.append(tt)
        
        
     
print(ppk)
len(pokemon_totals) # you should see 18
pokemon_totals

{'Electric': 23, 'Bug': 55, 'Poison': 47, 'Dark': 41, 'Psychic': 52, 'Fire': 36, 'Rock': 49, 'Dragon': 39, 'Steel': 44, 'Flying': 99, 'Water': 67, 'Fighting': 33, 'Ice': 25, 'Fairy': 25, 'Ghost': 36, 'Normal': 41, 'Ground': 54, 'Grass': 62}


[23, 55, 47, 41, 52, 36, 49, 39, 44, 99, 67, 33, 25, 25, 36, 41, 54, 62]

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [7]:
# Your code here
st.f_oneway(*pokemon_totals)

ValueError: zero-dimensional arrays cannot be concatenated

#### Interpret the ANOVA test result. Is the difference significant?

In [ ]:
# Your comment here